<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/Parquet_Files_Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-09-11 10:26:35.607114+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


For a more detailed exploration of Pivot Tables please look at this [Notebook : Analysis of Retail Sales Data](https://github.com/Praxis-QR/PythonForManagers/blob/main/Retail_Sales_Data_Analysis.ipynb)

#SuperStore Data
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [2]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [12]:
!tail SS_Orders.csv

9985,CA-2015-100251,17/05/2015,23/05/2015,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-LA-10003766,Office Supplies,Labels,Self-Adhesive Removable Labels,31.5,10,0,15.12
9986,CA-2015-100251,17/05/2015,23/05/2015,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-SU-10000898,Office Supplies,Supplies,"Acme Hot Forged Carbon Steel Scissors with Nickel-Plated Handles, 3 7/8"" Cut, 8""L",55.6,4,0,16.124
9987,CA-2016-125794,29/09/2016,03/10/2016,Standard Class,ML-17410,Maris LaWare,Consumer,United States,Los Angeles,California,90008,West,TEC-AC-10003399,Technology,Accessories,Memorex Mini Travel Drive 64 GB USB 2.0 Flash Drive,36.24,1,0,15.2208
9988,CA-2017-163629,17/11/2017,21/11/2017,Standard Class,RA-19885,Ruben Ausman,Corporate,United States,Athens,Georgia,30605,South,TEC-AC-10001539,Technology,Accessories,Logitech G430 Surround Sound Gaming Headset with Dolby 7.1 Technology,79.99,1,0,28.

In [5]:
import numpy as np
import pandas as pd

#Read CSV into Pandas Dataframe

In [6]:
dfSS = pd.read_csv('SS_Orders.csv', delimiter = ',')
dfSS.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [15]:
dfSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

##Comma, Double Quotes look good

In [11]:
dfSS.iloc[9993]['Product Name']

'Acco 7-Outlet Masterpiece Power Center, Wihtout Fax/Phone Line Protection'

In [13]:
dfSS.iloc[9992]['Product Name']

'It\'s Hot Message Books with Stickers, 2 3/4" x 5"'

##Date looks Bad, need to be fixed

In [18]:
#
# what is the first and last order date ?
# we have four years of data, but date format changes ...
dfSS['Order Date'].describe
#df['ship_date'].describe

<bound method NDFrame.describe of 0       08/11/2016
1       08/11/2016
2       12/06/2016
3       11/10/2015
4       11/10/2015
           ...    
9989    21/01/2014
9990    26/02/2017
9991    26/02/2017
9992    26/02/2017
9993    04/05/2017
Name: Order Date, Length: 9994, dtype: object>

In [21]:
# Some of the dates are string like dd/mm/yyyy while others are dd-mm-yyyy
# this will cause problems in converting string to real dates
# So we move all dd-mm-yyyy to dd/mm/yyyy -- could have been the other way round also
#
dfSS['Order Date'] = dfSS['Order Date'].str.replace('-','/')
dfSS['Ship Date'] = dfSS['Ship Date'].str.replace('-','/')

In [22]:
# convert all dates to dd/mm/yyyy format
#
#df['ship_date'] = pd.to_datetime(df['ship_date']).dt.strftime('%d/%m/%Y')
#df['order_date'] = pd.to_datetime(df['order_date']).dt.strftime('%d/%m/%Y')
#
dfSS['Ship Date'] = pd.to_datetime(dfSS['Ship Date'],format='%d/%m/%Y')
dfSS['Order Date'] = pd.to_datetime(dfSS['Order Date'],format='%d/%m/%Y')
#df['order_date'].describe

In [23]:
dfSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [38]:
#dfSS_2.info()

In [92]:
dfSS.to_csv('SS_Orders_Clean.csv', index=False)

In [93]:
#dfSS.to_csv('SS_Orders_Clean_temp.csv', index=False)
#!head SS_Orders_Clean_temp.csv

#Pandas Dataframe to Parquet Format
Ref https://mungingdata.com/pyarrow/parquet-metadata-min-max-statistics/

In [24]:
dfSS.to_parquet('SS_Orders.parquet')

In [32]:
parquet_file = pq.ParquetFile('SS_Orders.parquet')

In [33]:
parquet_file.metadata

  created_by: parquet-cpp-arrow version 9.0.0
  num_columns: 21
  num_rows: 9994
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 10777

In [34]:
parquet_file.metadata.row_group(0)

  num_columns: 21
  num_rows: 9994
  total_byte_size: 650681

In [36]:
parquet_file.metadata.row_group(0).column(1)

  file_offset: 112125
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 9994
  path_in_schema: Order ID
  is_stats_set: True
  statistics:
      has_min_max: True
      min: CA-2014-100006
      max: US-2017-169551
      null_count: 0
      distinct_count: 0
      num_values: 9994
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('RLE_DICTIONARY', 'PLAIN', 'RLE')
  has_dictionary_page: True
  dictionary_page_offset: 57750
  data_page_offset: 95899
  total_compressed_size: 54375
  total_uncompressed_size: 106440

In [37]:
parquet_file.metadata.row_group(0).column(20)

  file_offset: 459992
  file_path: 
  physical_type: DOUBLE
  num_values: 9994
  path_in_schema: Profit
  is_stats_set: True
  statistics:
      has_min_max: True
      min: -6599.978
      max: 8399.976
      null_count: 0
      distinct_count: 0
      num_values: 9994
      physical_type: DOUBLE
      logical_type: None
      converted_type (legacy): NONE
  compression: SNAPPY
  encodings: ('RLE_DICTIONARY', 'PLAIN', 'RLE')
  has_dictionary_page: True
  dictionary_page_offset: 398914
  data_page_offset: 443641
  total_compressed_size: 61078
  total_uncompressed_size: 74893

#Parquet to Pandas Dataframe

In [40]:
dfSS_2 = pd.read_parquet('SS_Orders.parquet')
dfSS_2.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [41]:
dfSS_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

#PyArrow

In [28]:
import pyarrow.csv as pv
import pyarrow.parquet as pq

In [42]:
table = pv.read_csv('SS_Orders_Clean.csv')

In [43]:
table_df = table.to_pandas()
table_df.head()

,,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [45]:
# Date is still not clean
table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0                  9994 non-null   int64  
 1   Row ID         9994 non-null   int64  
 2   Order ID       9994 non-null   object 
 3   Order Date     9994 non-null   object 
 4   Ship Date      9994 non-null   object 
 5   Ship Mode      9994 non-null   object 
 6   Customer ID    9994 non-null   object 
 7   Customer Name  9994 non-null   object 
 8   Segment        9994 non-null   object 
 9   Country        9994 non-null   object 
 10  City           9994 non-null   object 
 11  State          9994 non-null   object 
 12  Postal Code    9994 non-null   int64  
 13  Region         9994 non-null   object 
 14  Product ID     9994 non-null   object 
 15  Category       9994 non-null   object 
 16  Sub-Category   9994 non-null   object 
 17  Product Name   9994 non-null   object 
 18  Sales   

In [46]:
#
# what is the first and last order date ?
# we have four years of data, but date format changes ...
table_df['Order Date'].describe
#df['ship_date'].describe

<bound method NDFrame.describe of 0       2016-11-08
1       2016-11-08
2       2016-06-12
3       2015-10-11
4       2015-10-11
           ...    
9989    2014-01-21
9990    2017-02-26
9991    2017-02-26
9992    2017-02-26
9993    2017-05-04
Name: Order Date, Length: 9994, dtype: object>

In [49]:
# convert all dates to dd/mm/yyyy format
#
#df['ship_date'] = pd.to_datetime(df['ship_date']).dt.strftime('%d/%m/%Y')
#df['order_date'] = pd.to_datetime(df['order_date']).dt.strftime('%d/%m/%Y')
#
table_df['Ship Date'] = pd.to_datetime(table_df['Ship Date'],format='%Y-%m-%d')
table_df['Order Date'] = pd.to_datetime(table_df['Order Date'],format='%Y-%m-%d')

In [50]:
table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0                  9994 non-null   int64         
 1   Row ID         9994 non-null   int64         
 2   Order ID       9994 non-null   object        
 3   Order Date     9994 non-null   datetime64[ns]
 4   Ship Date      9994 non-null   datetime64[ns]
 5   Ship Mode      9994 non-null   object        
 6   Customer ID    9994 non-null   object        
 7   Customer Name  9994 non-null   object        
 8   Segment        9994 non-null   object        
 9   Country        9994 non-null   object        
 10  City           9994 non-null   object        
 11  State          9994 non-null   object        
 12  Postal Code    9994 non-null   int64         
 13  Region         9994 non-null   object        
 14  Product ID     9994 non-null   object        
 15  Category       9994 n

In [51]:
pq.write_table(table, 'SS_Orders_2.parquet')

#Spark

In [94]:
!head SS_Orders_Clean.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Unite

In [95]:
pdfSS = pd.read_csv('SS_Orders_Clean.csv')
pdfSS.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object

In [72]:
#pdfSS.to_csv('SS_Orders_Clean2.csv')

In [96]:
!pip3 -q install pyspark
from pyspark.sql import SparkSession
#spark9 = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
spark9 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()

In [97]:
!tail SS_Orders_Clean.csv

9985,CA-2015-100251,2015-05-17,2015-05-23,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-LA-10003766,Office Supplies,Labels,Self-Adhesive Removable Labels,31.5,10,0.0,15.12
9986,CA-2015-100251,2015-05-17,2015-05-23,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-SU-10000898,Office Supplies,Supplies,"Acme Hot Forged Carbon Steel Scissors with Nickel-Plated Handles, 3 7/8"" Cut, 8""L",55.6,4,0.0,16.124
9987,CA-2016-125794,2016-09-29,2016-10-03,Standard Class,ML-17410,Maris LaWare,Consumer,United States,Los Angeles,California,90008,West,TEC-AC-10003399,Technology,Accessories,Memorex Mini Travel Drive 64 GB USB 2.0 Flash Drive,36.24,1,0.0,15.2208
9988,CA-2017-163629,2017-11-17,2017-11-21,Standard Class,RA-19885,Ruben Ausman,Corporate,United States,Athens,Georgia,30605,South,TEC-AC-10001539,Technology,Accessories,Logitech G430 Surround Sound Gaming Headset with Dolby 7.1 Technology,79.99,1

In [98]:
sdfSS = spark9.read.csv('SS_Orders_Clean.csv', inferSchema=True,header = True, quote = '"')

In [99]:
#Has not been inferred correctly!
sdfSS.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('Order Date', 'date'),
 ('Ship Date', 'date'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'string'),
 ('Quantity', 'string'),
 ('Discount', 'string'),
 ('Profit', 'double')]

In [100]:
#we remove the CRLF character from the end of the row if it exists
!sed 's/\r//g' /content/SS_Orders_Clean.csv > SSdatafile.csv
# remove the first line containing headers from the file
#!sed -i -e "1d" datafile.csv
!head SSdatafile.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Unite

In [101]:
# remove , and " from data
import pandas as pd
p_df = pd.read_csv('SSdatafile.csv')
#p_df = pd.read_csv('datafile.csv')
p_df2 = p_df.replace(',', ' ', regex=True)
p_df3 = p_df2.replace('"', '', regex=True)
p_df3.to_csv('SSdatafile2.csv',index=False,sep=',')
!head SSdatafile2.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs  Rounded Back,731.94,3,0.0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United 

In [108]:
data_file = 'SSdatafile2.csv'
sdfSS2 = spark9.read.csv(data_file,inferSchema=True, header=True)
sdfSS2.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('Order Date', 'date'),
 ('Ship Date', 'date'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'double'),
 ('Quantity', 'int'),
 ('Discount', 'double'),
 ('Profit', 'double')]

##Spark SQL

In [104]:
spark9.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [109]:
#air_quality_sdf.createOrReplaceTempView("air_quality_sdf")
sdfSS2.createOrReplaceTempView("SS_TempTable")

In [106]:
spark9.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |ss_temptable|       true|
+---------+------------+-----------+



In [112]:
spark9.sql('select * from ss_temptable limit 5').show()

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [113]:
sdfSS2A = sdfSS2.withColumnRenamed("Order Date","OrderDate") \
    .withColumnRenamed("Ship Date","ShipDate")
#sdfSS2A.printSchema()
sdfSS2A.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('OrderDate', 'date'),
 ('ShipDate', 'date'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'double'),
 ('Quantity', 'int'),
 ('Discount', 'double'),
 ('Profit', 'double')]

In [114]:
sdfSS2A.createOrReplaceTempView("SS_TempTable2")

In [116]:
sql_drop_table = """
drop table if exists praxisdb.ss_orders_sparkhive
"""

sql_drop_database = """
drop database if exists praxisdb cascade
"""

sql_create_database = """
create database if not exists praxisdb
location '/content/praxisdb/'
"""

sql_create_table = """
create table if not exists praxisdb.ss_orders_sparkhive
using parquet
as select *
from ss_temptable2
"""

#as select to_timestamp(date) as date_parsed, *

print("dropping database...")
result_drop_db = spark9.sql(sql_drop_database)

print("creating database...")
result_create_db = spark9.sql(sql_create_database)

print("dropping table...")
result_droptable = spark9.sql(sql_drop_table)

print("creating table...")
result_create_table = spark9.sql(sql_create_table)

dropping database...
creating database...
dropping table...
creating table...


In [119]:
spark9.sql("select * from praxisdb.ss_orders_sparkhive").show(5)
#.select("date_parsed", "O_3").show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID| OrderDate|  ShipDate|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

##Parquet to Pandas again

In [120]:
dfSS_3 = pd.read_parquet('/content/praxisdb/ss_orders_sparkhive/part-00000-8ecf12c5-158f-4cd0-8918-378834a54762-c000.snappy.parquet')

In [121]:
dfSS_3.head()

,Row ID,Order ID,OrderDate,ShipDate,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs ...,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [122]:
dfSS_3.dtypes

Row ID             int32
Order ID          object
OrderDate         object
ShipDate          object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int32
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int32
Discount         float64
Profit           float64
dtype: object

#Hive to Pandas

In [127]:
!pip install impyla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for thrift: filename=thrift-0.16.0-cp310-cp310-linux_x86_64.whl size=373871 sha256=7a172cb02db00b48cbfb6f570267a1efb3b18778c9408be1c09e4b0a2284a047
  Stored in directory: /root/.cache/pip/wheels/52/f8/d2/acfd995e8247eb0cad372fa6a640a5fcf279ab2ed7c5c4490e
  Created wheel for pure-sasl: filename=pure_sasl-0.6.2-py3-none-any.whl size=11423 sha256=54c2f7b9378a7403cb9c31a01a3320ef5c4ba32afc0512d82e6de067ac92e67d
  Stored in directory: /root/.cache/pip/wheels/57/7c/93/062238b0a68efe214024ca178233f248971045db1033c96a52
Successfully built thrift pure-sasl


In [1]:
from impala.dbapi import connect
from impala.util import as_pandas

In [3]:
hive_conn = connect(host='localhost', port=21050, database='praxisdb', auth_mechanism='PLAIN')

ERROR:thrift.transport.TSocket:Could not connect to any of [('127.0.0.1', 21050), ('::1', 21050, 0, 0)]


TTransportException: ignored

In [ ]:


with hive_conn.cursor() as c:
    c.execute('SELECT * FROM analytics.pandas_spark_hive LIMIT 100')
    results = c.fetchall()

with hive_conn.cursor() as c:
    c.execute('SELECT * FROM analytics.pandas_spark_hive LIMIT 100')
    results_df = as_pandas(c)

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)